# Decision Trees

Decision trees use a tree structure to represent a number of possible decision paths and an outcome for each path.  Similar to the game 20 Questions.  For example:

- "I am thinking of an animal"
- "Does it have more than five legs?"
- "No"
- "Is it delicious?"
- "No"
- "Does it appear on the back of the Australian five-cent coin?"
- "Yes"
- "Is it an echidna?"
- "Yes"

This corresponds to the path:

"Not more than 5 legs" --> "Not Delicious" --> "On the 5 cent coin" --> "Echidna"

Decision trees are very easy to understand and interpret.  The process buy which they reach a prediction is completely transparent.  Unlike the other models we've looked at so far, decision trees can easily handle a mix of numeric and categorical attributes and can even classify data for which attributes are missing.

At the same time, finding optimal decision tree for a set of training data is computationally a very hard problem.  More important, it is very easy (and bad) to build decision trees that are overfitted to the training data, and that don't generalize well to unseen data.  We'll look at ways to address this.  Most people divide decision trees into *classification trees* (which produce categorical outputs), and *regression trees* (which produce numeric outputs).

In order to build a decision tree, we will need to decide what questions to ask and in what order.  At each stage of the tree there are some possibilities we've eliminated and some that we haven't.  After learning that an animal doesn't have more than five legs, we've eliminated the possibility that it's a grasshopper.  We haven't eliminated the possibility that it's a duck.  Every possible question partitions the remaining possibilities according to their answers.

Ideally, we'd like to choose questions whose answers give a lot of information about what our tree should predict.  If there's a single yes/no question for which "yes" answers always correspond to `True` outputs and "no" answers to `False` outputs, this would be an awesome question to pick.  Conversely, a yes/no question for which neither answer gives you much new information about what the prediction should be is probably not a good choice.  We capture this notion of "how much information" with *entropy*.  Entropy typically is defined as disorder, but we use it in stats to represent the uncertainty associated with the data.

Imagine that we have a set $ S $ of data, each member of which is labeled as belonging to one of a finite number of classes $ C_1, ..., C_n $.  If all the data points belong to a single class, then there is no real uncertainty, which means we'd like there to be low entropy.  If the data points are evenly spread across the classes, there is a lot of uncertainty and we'd like there to be high entropy.  In math terms, if $ p_i $ is the proportion of data labeled as calss $ c_i $, we define the entropy as:

$$ H(S) = - p_1 log_2 P_1 - ... - P_n log_2 P_n $$

with the standard convention that 0 log 0 = 0.  $ p_i log_2 P_i $ is non-negative and is close to zero precisely when $ p_i $ is either close to zero or close to one.  This means the entropy will be small when every $ p_i $ is close to 0 or 1 (ie when most of the data is in a single class), and it will be larger when many of the $ p_i $'s are not close to 0 (ie when the data is spread across multiple classes).  This is exactly the behavior we desire.  Functionally, it looks like:

In [1]:
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

Our data will consist of pairs (input, label), which means that we'll need to compute the class probabilities ourselves.  Observe that we don't actually care which label is associated with each probability, only what the probabilities are:

In [2]:
def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

We've now got functions to compute the entropy (or uncertainty) of a single set of labeled data.  Now, each stage of a decision tree involves asking a question whose answer partitions data into one or more subsets.  Correspondingly, we'd like some notion of the entropy that results from partitioning a set of data in a certain way.  We want a partition to have low entropy if it splits the data into subsets that themselves have low entropy, and high entropy if it contains subsets that are large and have high entropy.

Mathematically, if we partition our data $ S $ into subsets $ S_1, ..., S_m $ containing proprtions $ q_1, ..., q_m $ of the data, then we compute the entropy of the partition as a weighted sum:

$$ H = q_1H(S_1) + ... + q_mH(S_m) $$

Which we can implement as:

In [3]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

As inputs for the creation of a decision tree, we'll use a `dict` of candidate attributes, and each label is either `True` or `False`:

In [4]:
inputs = [
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
    ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
    ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
    ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
    ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
]

Our tree will consist of *decision nodes* and *leaf nodes*.  We will build it using the ID3 algorithm, which operates like so:

- If the data all have the same label, then create a leaf node that predicts that label and then stop.
- If the list of attributes is empty, then create a leaf node that predicts the most common label and then stop.
- Otherwise, try partitioning the data by each of the attributes.
- Choose the partition with the lowest partition entropy.
- Add a decision node based on the chosen attribute.
- Recur on each partitioned subset using the remainder attributes.

This is what's known as a "greedy" algorithm because at each step, it chooses the most immediately best option.  Given a data set, there may be a better tree with a worse-looking first move.  If so, this algorithm won't find it.  Nonetheless, it is a relatively easy to understand and implement:

In [6]:
def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute
    each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs,attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


The lowest entropy comes from splitting on `level`, so we'll need to make a subtree for each possible `level` value.  Every `Mid` candidate is labeled `True`, which means that the `Mid` subtree is simply a leaf node predicting `True`.  For `Senior` canidates, we have a mix of `True` and `False`, so we need to split again:

In [7]:
senior_inputs = [(input, label) for input, label in inputs if input['level'] == "Senior"]

for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


This shows us that our next split should be on `tweets`, which results in a zero entropy partition.  For these Senior level candidates, "yes" tweets always result in `True` while "no" tweets always result in `False`.  Finally, if we do the same thing for the `Junior` candidates, we end up splitting on `phd`, after which we find that no PhD always results in `True` and PhD always results in `False`.  Now that we've seen how the algo works, let's implement it more generally.  This means we need to decide how we want to represent trees.  We'll use pretty much the most lightweight representation possible.  We define a tree to be one of the following:

- `True`
- `False`
- a `tuple (attribute, subtree_dict)`

Here, `True` represents a leaf node that returns `True` for any input.  `False` represents a leaf node that returns `False` for any input, and a tuple represents a decision node that, for any input, finds its `attribute` value, and classifies the input using the corresponding subtree.  With this representation, our hiring tree would look like:

In [11]:
def classify(tree, input):
    """classify the input using the given decision tree"""

    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree

    # otherwise find the correct subtree
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree

    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input

def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf

    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf

    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

tree = build_tree_id3(inputs)
print(tree)
print("Junior / Java / tweets / no phd", classify(tree,
    { "level" : "Junior",
      "lang" : "Java",
      "tweets" : "yes",
      "phd" : "no"} ))

print("Junior / Java / tweets / phd", classify(tree,
    { "level" : "Junior",
             "lang" : "Java",
             "tweets" : "yes",
             "phd" : "yes"} ))

print("Intern", classify(tree, { "level" : "Intern" } ))
print("Senior", classify(tree, { "level" : "Senior" } ))

('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True}), None: True})
Junior / Java / tweets / no phd True
Junior / Java / tweets / phd False
Intern True
Senior False


Given how closely decision trees can fit themselves to their training data, it's not surprising that they have a tendency to overfit.  One way to avoid this is a technique called *random forests*, in which we build multiple decision trees and let them vote on how to classify inputs:

In [13]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

